In [1]:
#load packages
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np

C:\Users\Ayeye\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#load data
data=sm.datasets.get_rdataset('epil',package='MASS').data

In [3]:
#examine what columns exist in the data
print(data.shape)
data.head()

(236, 9)


,y,trt,base,age,V4,subject,period,lbase,lage
0,5,placebo,11,31,0,1,1,-0.756354,0.114204
1,3,placebo,11,31,0,1,2,-0.756354,0.114204
2,3,placebo,11,31,0,1,3,-0.756354,0.114204
3,3,placebo,11,31,1,1,4,-0.756354,0.114204
4,3,placebo,11,30,0,2,1,-0.756354,0.081414


In [4]:
#look at mean number of seizures in two-week time frame by treatment and period
pd.pivot_table(data,values='y', index=['period'],
                    columns=['trt'], aggfunc=np.mean)

trt,placebo,progabide
period,,
1,9.357143,8.580645
2,8.285714,8.419355
3,8.714286,8.129032
4,7.964286,6.709677


In [5]:
#mean number of seizures in two-week time frame by treatment and subject
pd.pivot_table(data,values='y', index=['subject'],
                    columns=['period'], aggfunc=np.mean)

period,1,2,3,4
subject,,,,
1,5,3,3,3
2,3,5,3,3
3,2,4,0,5
4,4,4,1,4
5,7,18,9,21
6,5,2,8,7
7,6,4,0,2
8,40,20,21,12
9,5,6,6,5


In [6]:
#define GEE parameters
family=sm.families.Poisson() #count data
cov_str=sm.cov_struct.Exchangeable() #data-derived covariance structure

In [7]:
#create GEE model predicting seizures based on treatment, baseline measurement,
#patient, and time period
model=smf.gee("y ~ trt + base","subject",data,cov_struct=cov_str,
              family=family)
results=model.fit()

In [8]:
#examine resulting model
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               GEE Regression Results                              
===================================================================================
Dep. Variable:                           y   No. Observations:                  236
Model:                                 GEE   No. clusters:                       59
Method:                        Generalized   Min. cluster size:                   4
                      Estimating Equations   Max. cluster size:                   4
Family:                            Poisson   Mean cluster size:                 4.0
Dependence structure:         Exchangeable   Num. iterations:                     2
Date:                     Wed, 26 Oct 2022   Scale:                           1.000
Covariance type:                    robust   Time:                         17:21:04
====================================================================================
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.2781      0.133      9.576      0.000       1.016       1.540
trt[T.progabide]    -0.2231      0.175     -1.271      0.204      -0.567       0.121
base                 0.0218      0.001     20.806      0.000       0.020       0.024
==============================================================================
Skew:                          3.8238   Kurtosis:                      28.0529
Centered skew:                 2.7597   Centered kurtosis:             21.9865
==============================================================================
"""